This notebook registers a custom model LLM to be used in a GenAI Playground.

## Prerequisites

The notebook assumes you already have a deployed custom model with the Text Generation output type.

## Feature Flags

Make sure the following feature flags are enabled:

   * Enable Document Ingest
   * Enable Custom Inference Models
   * Enable the Injection of Runtime Parameters for Custom Models
   * Enable Public Network Access for all Custom Models
   * Enable Monitoring Support for Generative Models
   * Enable GenAI Experimentation

## Configuration

   * Set the `DATAROBOT_API_TOKEN` environment variable in this notebook to the value of a manually created DataRobot API Key.
   * Set the `BYO_LLM_DEPLOYMENT_ID` environment variable in this notebook to the value of your custom LLM deployment.

In [ ]:
import os
import requests
import time

In [ ]:
byo_llm_deployment_id = os.environ["BYO_LLM_DEPLOYMENT_ID"]

genai_endpoint_url = "https://staging.datarobot.com/api-gw/genai"
datarobot_api_key = os.environ["DATAROBOT_API_TOKEN"]

genai_client = requests.session()
genai_client.headers.update({"Authorization": f"Bearer {datarobot_api_key}"})

## Validate the custom model deployment

Request a new custom model validation

In [ ]:
validation_payload = {
    "promptColumnName": "promptText",
    "targetColumnName": "resultText",
    "deploymentId": byo_llm_deployment_id,
}
response = genai_client.post(f"{genai_endpoint_url}/customModelLLMValidations/", json=validation_payload)
status_tracking_url = response.headers.get('Location')

print(f"Response: {response.status_code}")
print(f"Status tracking URL: {status_tracking_url}")

Response: 202
Status tracking URL: https://staging.datarobot.com/api-gw/genai/status/b0521405-2958-4f25-b5c3-ffd9417e5522/


Wait for the validation job to finish

In [ ]:
validation_id = None

# Waiting timeout = 60 seconds
for _ in range(60):
    status_response = genai_client.get(status_tracking_url, allow_redirects=False)

    if status_response.status_code == 303:
        validation_id = response.json()["id"]
        validation = genai_client.get(f"{genai_endpoint_url}/customModelLLMValidations/{validation_id}/").json()
        validation_status = validation["validationStatus"]

        if validation_status == "PASSED":
            print(f"Successful validation ID: {validation['id']}")
            break
        else:
            raise RuntimeError(f"Expected a successful validation, got: {validation_status}")

    time.sleep(1)

if not validation_id:
    raise RuntimeError("Timed out waiting for custom model validation to succeed")

Successful validation ID: 6543e385b8ed06236c7b63ae
